# GFDL FLOR Historical

## Loads in all monthly forecasts of sea ice extent
## Regrids to polar stereographic,
## Saves to a netcdf file

In [1]:
# Standard Imports
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import numpy as np
import xarray as xr
import xesmf as xe
import os
import glob
import seaborn as sns
# ESIO Imports
import esio

In [2]:
# General plotting settings
sns.set_style('whitegrid')
sns.set_context("talk", font_scale=1.5, rc={"lines.linewidth": 2.5})

In [3]:
# Data dir
data_dir = r'/home/disk/sipn/nicway/data/model/nmme/reforecast/native'
data_out = r'/home/disk/sipn/nicway/data/model/nmme/reforecast/sipn_nc'
grid_file = r'/home/disk/sipn/nicway/data/grids/GFDLFLOR_gridinfo.nc' # Correct GFDL grid info

In [4]:
# Load in correct GFDL grid info and format
grid = xr.open_dataset(grid_file)
n_lat = grid.grid_center_lat.values.reshape(tuple(np.flip(grid.grid_dims.T.values,0))) # Reshape
n_lat = np.rad2deg(n_lat)
n_lon = grid.grid_center_lon.values.reshape(tuple(np.flip(grid.grid_dims.T.values,0))) # Reshape
n_lon = np.rad2deg(n_lon)

In [5]:
# Open all ensemble members as a dataset
year= '_201' # Specific year, or 
ds_list = []
for e in np.arange(1,13,1):
    ensm_name = 'r'+str(e)+'i1p1'
    cfiles = os.path.join(data_dir, '*sic*'+ensm_name+'*'+year+'*.nc')
    ds_list.append(esio.open_1_member(cfiles, e))
ds_all = xr.concat(ds_list, dim='ensemble')

/home/disk/sipn/nicway/anaconda3/envs/xesmf/lib/python3.6/site-packages/xarray/conventions.py:1037: UserWarning: CF decoding is overwriting dtype on variable 'sic'
  .format(name))


In [6]:
# Fix GFDL grid
# Files comes with incorrect lat/lon at poles. Use correct separte file to update coords
ds_all.rename({'lat':'nj', 'lon':'ni'}, inplace=True); # Rename wrong-lat/lon to indices
ds_all.coords['nj'] = xr.DataArray(np.arange(0,ds_all.nj.size,1), dims=('nj')) # Make indices
ds_all.coords['ni'] = xr.DataArray(np.arange(0,ds_all.ni.size,1), dims=('ni'))
ds_all.coords['lat'] = xr.DataArray(n_lat, dims=('nj','ni'), coords={'nj':ds_all.nj, 'ni':ds_all.ni}) #Add correct lat/lon
ds_all.coords['lon'] = xr.DataArray(n_lon, dims=('nj','ni'), coords={'nj':ds_all.nj, 'ni':ds_all.ni})

In [7]:
# Average over all sea ice categories 
sic = ds_all.sic.sum(dim='ct', skipna=False)

In [8]:
# Get info about target grid
ds_out = esio.get_stero_N_grid()
ds_out

<xarray.Dataset>
Dimensions:  (x: 304, y: 448)
Dimensions without coordinates: x, y
Data variables:
    lat      (x, y) float64 31.1 31.25 31.4 31.55 31.69 31.84 31.99 32.13 ...
    lon      (x, y) float64 168.3 168.4 168.5 168.7 168.8 168.9 169.0 169.1 ...

In [9]:
# Calculate regridding matrix
regridder = xe.Regridder(sic, ds_out, 'bilinear', periodic=True)

/home/disk/sipn/nicway/anaconda3/envs/xesmf/lib/python3.6/site-packages/xesmf/backend.py:36: UserWarning: Input array is not F_CONTIGUOUS. Will affect performance.
  warnings.warn("Input array is not F_CONTIGUOUS. "


Create weight file: bilinear_200x360_304x448_peri.nc


In [10]:
# Regrid
sic_out = regridder(sic)

In [11]:
# Add dropped coords
sic_out['fore_time'] = sic.fore_time

In [13]:
# # Save regridded to netcdf file
f_out = os.path.join(data_out, 'GFDL_FLORB01_Stereo.nc')
sic_out.to_netcdf(f_out)

In [14]:
# Clean up
regridder.clean_weight_file()  # clean-up

Remove file bilinear_200x360_304x448_peri.nc


In [15]:
sic.init_time

<xarray.DataArray 'init_time' (init_time: 54)>
array(['2010-01-01T00:00:00.000000000', '2010-02-01T00:00:00.000000000',
       '2010-03-01T00:00:00.000000000', '2010-04-01T00:00:00.000000000',
       '2010-05-01T00:00:00.000000000', '2010-06-01T00:00:00.000000000',
       '2010-07-01T00:00:00.000000000', '2010-08-01T00:00:00.000000000',
       '2010-09-01T00:00:00.000000000', '2010-10-01T00:00:00.000000000',
       '2010-11-01T00:00:00.000000000', '2010-12-01T00:00:00.000000000',
       '2011-01-01T00:00:00.000000000', '2011-02-01T00:00:00.000000000',
       '2011-03-01T00:00:00.000000000', '2011-04-01T00:00:00.000000000',
       '2011-05-01T00:00:00.000000000', '2011-06-01T00:00:00.000000000',
       '2011-07-01T00:00:00.000000000', '2011-08-01T00:00:00.000000000',
       '2011-09-01T00:00:00.000000000', '2011-10-01T00:00:00.000000000',
       '2011-11-01T00:00:00.000000000', '2011-12-01T00:00:00.000000000',
       '2012-01-01T00:00:00.000000000', '2012-02-01T00:00:00.000000000',
    

# Plotting

In [ ]:
# # Set up plotting info
# cmap_sic = matplotlib.colors.ListedColormap(sns.color_palette("Blues", 10))
# cmap_sic.set_bad(color = 'lightgrey')

In [ ]:
# # Plot original projection
# plt.figure(figsize=(20,10))
# ax1 = plt.axes(projection=ccrs.PlateCarree())
# # ds_p = sic.mean(dim='init_time').mean(dim='fore_time').mean(dim='ensemble')
# ds_p = sic.isel(init_time=0).isel(fore_time_i=6).mean(dim='ensemble')
# ds_p.plot.pcolormesh(ax=ax1, x='lon', y='lat', 
#                                  vmin=0, vmax=1,
#                                  cmap=matplotlib.colors.ListedColormap(sns.color_palette("Blues", 10)));
# ax1.set_extent([-180, 180, -90, 90], crs=ccrs.PlateCarree())
# gl = ax1.gridlines(crs=ccrs.PlateCarree(), linestyle='-')
# gl.xlabels_bottom = True
# gl.ylabels_left = True
# gl.xformatter = LONGITUDE_FORMATTER
# gl.yformatter = LATITUDE_FORMATTER
# ax1.coastlines(linewidth=0.75, color='black', resolution='50m');

In [ ]:
# # Plot SIC on target projection
# plt.figure(figsize=(10,8));
# ax1 = plt.axes(projection=ccrs.NorthPolarStereo(central_longitude=0))
# # ds_p2 = sic_out.mean(dim='init_time').mean(dim='fore_time').mean(dim='ensemble')
# ds_p2 = sic_out.isel(init_time=0).isel(fore_time_i=6).mean(dim='ensemble')
# ds_p2.plot.pcolormesh(ax=ax1, x='lon', y='lat', 
#                                      transform=ccrs.PlateCarree(),
#                                      cmap=cmap_sic)
# # Add coastlines and meridians/parallels 
# ax1.coastlines(linewidth=0.75, color='black', resolution='50m')
# ax1.gridlines(crs=ccrs.PlateCarree(), linestyle='-')
# ax1.set_extent([-180, 180, 50, 90], crs=ccrs.PlateCarree())
# plt.tight_layout()